# Importing All Stuffs

In [11]:
import time
from bs4 import BeautifulSoup
from selenium import webdriver
import requests
from post_details import PostDetais
import json

# Class for collecting post contents

In [18]:
class PostDetais(object):
    def __init__(self, soup, link=None):
        self.page_soup = soup
        self.link = link

    def get_title(self):
        class_names = ['jz b ka kh ay',
                       'graf graf--h3 graf--leading graf--title',
                       'graf graf--h3 graf-after--figure graf--trailing graf--title']
        try:
            for my_tag in self.page_soup.find_all(True, {
                                                    'class': class_names}):
                title = my_tag.text
                return title
        except Exception as e:
            error_trace = {}
            error_trace["link"] = self.link
            error_trace["method"] = "get_title"
            error_trace["message"] = str(e)
            print(json.dumps(error_trace, indent=4))
        return ""

    def get_author_name(self):
        class_names = "ck cl cm cn co cp cq cr cs ct jm cw cx cy cz"
        try:
            for my_tag in self.page_soup.find_all(
                    class_=class_names):
                name = my_tag.text
                return name
        except Exception as e:
            error_trace = {}
            error_trace["link"] = self.link
            error_trace["method"] = "get_title"
            error_trace["message"] = str(e)
            print(json.dumps(error_trace, indent=4))
        return ""

    def get_date(self):
        class_names = 'ck cl cm cn co cp cq cr cs ct jm cw cx cy cz'
        try:
            for my_tag in self.page_soup.find_all(class_names):
                date_time = my_tag.text
                return date_time
        except Exception as e:
            error_trace = {}
            error_trace["link"] = self.link
            error_trace["method"] = "get_title"
            error_trace["message"] = str(e)
            print(json.dumps(error_trace, indent=4))
        return ""

    def get_read(self):
        try:
            for my_tag in self.page_soup.find_all(class_="readingTime"):
                read = my_tag.get('title')
                return read
        except Exception as e:
            error_trace = {}
            error_trace["link"] = self.link
            error_trace["method"] = "get_title"
            error_trace["message"] = str(e)
            print(json.dumps(error_trace, indent=4))
        return ""

    def get_upvote(self):
        class_names = 'u-relative u-background js-actionMultirecommendCount u-marginLeft5'
        try:
            for my_tag in self.page_soup.find_all('span', {
                'class': class_names}):
                upvotes = my_tag.text
                return upvotes
        except Exception as e:
            error_trace = {}
            error_trace["link"] = self.link
            error_trace["method"] = "get_upvote"
            error_trace["message"] = str(e)
            print(json.dumps(error_trace, indent=4))
        return ""

    def get_body(self):
        news_body = ""
        for paragraphs in self.page_soup.find_all(
                class_='graf graf--p graf-after--p'):
            news_body += paragraphs.text.rstrip().lstrip()
        return news_body

    def get_post_content(self):
        try:
            # class_="section-content"
            # we are using <article> tag as section-content class no longer works
            # everything is inside the article tag including title & author names
            for content in self.page_soup.find_all('article'):
                return content.text
        except Exception as e:
            error_trace = {}
            error_trace["link"] = self.link
            error_trace["method"] = "get_post_content"
            error_trace["message"] = str(e)
            print(json.dumps(error_trace, indent=4))
        return ""

    def get_response(self):
        class_names = 'button button--chromeless u-baseColor--buttonNormal u-marginRight12'
        try:
            class_names = 'button button--chromeless u-baseColor--buttonNormal u-marginRight12'
            for my_tag in self.page_soup.find_all('button', {
                'class': class_names}):
                res = my_tag.text
                return res
        except Exception as e:
            error_trace = {}
            error_trace["link"] = self.link
            error_trace["method"] = "get_response"
            error_trace["message"] = str(e)
            print(json.dumps(error_trace, indent=4))
        return ""

# Class for Scrapping pages

In [21]:
class MediumScrapper(object):
    def __init__(self, CHROME_DRIVER_PATH='/home/mrx/Downloads/chromedriver'):
        self.CHROME_DRIVER_PATH = CHROME_DRIVER_PATH
        content = self.get_intial_content()
        self.parsed_data = BeautifulSoup(content, 'lxml')

    def get_intial_content(self,
                           base_url="https://medium.com/search?q=ethereum"):
        driver = webdriver.Chrome(self.CHROME_DRIVER_PATH)
        driver.get(base_url)
        scrolls = 1
        while scrolls > 0:
            driver.execute_script(
                "window.scrollTo(0, document.body.scrollHeight-1000);")
            time.sleep(10)
            scrolls -= 1
        # driver.implicitly_wait(30)
        time.sleep(10)
        content = driver.execute_script(
            "return document.documentElement.outerHTML")
        driver.quit()
        return content

    def get_post_links(self):
        links = []
        class_names = "button button--smaller button--chromeless u-baseColor--buttonNormal"
        for my_tag in self.parsed_data.find_all(class_=class_names):
            links.append(my_tag.get('href'))
        return links

    def get_post_contents(self):
        links = self.get_post_links()
        data = []
        headers = requests.utils.default_headers()
        headers.update({
            'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0',
        })
        for link in links:
            try:
                print("Scrapping link: {}".format(link))
                time.sleep(15)
                request_link = requests.get(link, headers=headers)
                request_content = BeautifulSoup(request_link.content,
                                                'html.parser')
                post_details = PostDetais(request_content, link)
                post_title = post_details.get_title()
                author_name = post_details.get_author_name()
                post_date = post_details.get_date()
                post_readtime = post_details.get_read()
                post_upvotes = post_details.get_upvote()
                post_contents = post_details.get_post_content()
                post_responses = post_details.get_response()
                single_post = {
                    "title": post_title,
                    "author_name": author_name,
                    "link": link,
                    "post_date": post_date,
                    "readtime": post_readtime,
                    "upvotes": post_upvotes,
                    "content": post_contents,
                    "responses": post_responses
                }
                data.append(single_post)
            except Exception as e:
                print("Error in scrapping link: {}".format(link))
                print(str(e))
        return data

# Main Application

In [22]:
if __name__ == '__main__':
    scrapper = MediumScrapper()
    output_filename = 'test5.json'
    data = scrapper.get_post_contents()
    with open(output_filename, 'w') as fp:
        json.dump(data, fp)
    print("Check JSON file: {}".format(output_filename))
    print("Total posts: {}".format(len(data)))

Scrapping link: https://medium.com/@preethikasireddy/how-does-ethereum-work-anyway-22d1df506369?source=search_post---------0
Scrapping link: https://medium.com/hackernoon/neo-versus-ethereum-why-neo-might-be-2018s-strongest-cryptocurrency-79956138bea3?source=search_post---------1
Scrapping link: https://medium.com/free-code-camp/a-hacker-stole-31m-of-ether-how-it-happened-and-what-it-means-for-ethereum-9e5dc29e33ce?source=search_post---------2
Scrapping link: https://medium.com/hackernoon/ethereum-will-pass-bitcoin-in-2018-my-cryptocurrency-investment-portfolio-dcab52905bba?source=search_post---------3
Scrapping link: https://medium.com/hackernoon/cardano-ethereum-and-neo-killer-or-overhyped-and-overpriced-8fcd5f8abcdf?source=search_post---------4
Scrapping link: https://medium.com/loom-network/ethereum-will-be-the-backbone-of-the-new-internet-88718e08124f?source=search_post---------5
Scrapping link: https://blog.coinbase.com/ethereum-classic-etc-is-currently-being-51-attacked-33be13ce

# Testing the Results

In [23]:
file_name = "test5.json"
with open(file_name, "r") as f:
    data = json.load(f)

print(data[0]['title'])